## Introduction

There are lottery systems in many games. The simplest implementation is that one gives a fixed probability and draws the lottery with that probability. The number of trials for a win has a geometric distribution. This distribution has a drawback that the variance is very large especially when the probability is low. So we'd like to have some alternative systems to solve this problem.

In [1]:
import numpy as np
import plotly
import plotly.graph_objs as go
import random

from utils import *

### Let's first see how the geometric distribution looks like

In [2]:
def geometric(p):
    """Returns the procedure for the geometric distribution"""
    def procedure():
        return random.random() < p
    return procedure

In [3]:
geometric(0.1)()

False

In [4]:
geometric_dic = mc(geometric(0.01), simulation=1_000_000)

In [5]:
get_mean_std(geometric_dic)

(100.69267948847045, 99.16500640471219)

In [6]:
def theoretical_geometric_mean_std(p):
    mean = 1 / p
    var = (1 - p) / p / p
    return mean, np.sqrt(var)

In [7]:
theoretical_geometric_mean_std(0.01)

(100.0, 99.498743710662)

In [8]:
plot_pdf(geometric_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '1211e95d-d61d-4195-9e2c-8c66845b11fb',
 …

In [9]:
plot_cdf(geometric_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '217149b4-e2c4-4725-903e-4901d428e672',
 …

### For simple a geometric distribution, we see even though the mean is only 100, but there is still some probability to fail more than 300 trials

## A Better Solution

The challenging thing is how to make another probability distribution but keep the expectation unchanged. Suppose we'd like the expectation of every trial to be 0.01. When we fail for one trial, we don't want to lose completely but gain some value such as 0.005 which means in later trials it will generate the expectation of 0.005. 

Now we need to calculate the probability of winning and losing. Assume the winning probability is a, and losing probability is 1 - a. When win we gain 1, when lose we gain 0.005. So the "expectation" will be a + 0.005 * (1 - a), which should be equal to 0.01. So we can solve to get that a = 0.005.

So the initial probability is decreased, but later probability should raise, how? Take an example that we failed 60 trials, the gained value is accumulated to 0.3. If we win for the next trial, we consider that the value of 0.3 is used and it should be set back to zero, meaning that we only gain 0.7 since the gained value is decreased by 0.3. If the winning probability were still 0.005, we gained less, so the probability should increase. The expectation of the next trial is a * (1 - 0.3) + (1 - a) * 0.005. Again we can solve the value of a.

One more thing to notice is that when the accumulated gained value is more than 0.99. The solution gives a probability that is greater than 1. In fact even if we have probability of 1 to win, the gain is still less than 0.01 if the accumulated gain is reset to zero after win. So in this case, we should not set the accumulated gain to zero, we should first add 0.01 and then subtract 1 instead.

### Let's see how to implement it in general

In [10]:
def accumulated(p, x):
    """Create a procedure that has the expectation of p and the gaining value increase by x for every failed trial"""
    assert p >= x >= 0
    left = 0
    def procedure():
        nonlocal left
        if left + p < 1:
            # Note, p >= x and left + p < 1 guarantee 1 - left - x > 0
            r = (p - x) / (1 - left - x)
            win = random.random() < r
            if win:
                left = 0
            else:
                left += x
            return win
        else:
            left = left + p - 1
            return True
    return procedure

In [11]:
procedure = accumulated(0.01, 0.005)

In [12]:
acc_dic = mc(procedure, simulation=1_000_000)

In [13]:
get_mean_std(acc_dic)

(100.83390480032271, 57.958210414343334)

In [14]:
plot_pdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '23ab7838-07af-4114-8a3b-a9b1848a4f1a',
 …

In [15]:
plot_cdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '298ff436-b548-497b-9dcf-0123955f0729',
 …

### This seems to have a uniform distribution

In [16]:
procedure = accumulated(0.01, 0.003)

In [17]:
acc_dic = mc(procedure, simulation=1_000_000)

In [18]:
get_mean_std(acc_dic)

(100.28422425032595, 72.68558477131556)

In [19]:
plot_pdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '2722e03b-0b73-4945-b788-3f79c0f6f55c',
 …

In [20]:
plot_cdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'd1b5a3c5-1da5-4221-87a8-ded1e604b9db',
 …

In [21]:
procedure = accumulated(0.01, 0.007)

In [22]:
acc_dic = mc(procedure, simulation=1_000_000)

In [23]:
get_mean_std(acc_dic)

(100.70040281973817, 41.17190501731038)

In [24]:
plot_pdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '4366109b-9485-4697-b984-035a80856f57',
 …

In [25]:
plot_cdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '8b5a267a-cc47-41d0-b6c4-abc31c674a5e',
 …

From the three examples above, we see the variance is reduce and you will never fail more than 400 times.

## More complex methods

In the above method, when lose we gain a fixed value x. We can change it to make more complex methods. For example, gain x1 when the accumulated gain is less than 0.5 and gain x2 otherwise.

In [26]:
def accumulated_complex(p, x1, x2):
    assert p >= x1 >= 0 and p >= x2 >= 0
    left = 0
    def procedure():
        nonlocal left
        to_left = x1 if left < 0.5 else x2
        if left + p < 1:
            r = (p - to_left) / (1 - left - to_left)
            win = random.random() < r
            if win:
                left = 0
            else:
                left += to_left
            return win
        else:
            left = left + p - 1
            return True
    return procedure

In [27]:
procedure = accumulated_complex(0.01, 0.007, 0.003)

In [28]:
acc_dic = mc(procedure, simulation=1_000_000)

In [29]:
get_mean_std(acc_dic)

(99.78872255489021, 49.01473582853449)

In [30]:
plot_pdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '4102cc5f-cf65-4e7a-854c-5dafac76aa59',
 …

In [31]:
plot_cdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'fdf45476-a398-4c1e-b748-31c943308ee1',
 …

## Conclusion

We introduced a lottery method that has lower variances than geometric distributions and can also easily keep the gaining expectation unchanged.